In [75]:
import json, re
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen
from sklearn.feature_extraction import DictVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import *
from sklearn.metrics import accuracy_score, pairwise_distances
from sklearn.ensemble import RandomForestClassifier
# For transformations and predictions
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt


# For predictions
from sklearn.base import BaseEstimator
import scipy

In [74]:
with open('conditional_order.json", 'r') as f:
    conditional_order = json.load(f)
df = pd.DataFrame(conditional_order).set_index("מספר תיק")

SyntaxError: EOL while scanning string literal (<ipython-input-74-87df45802d2e>, line 1)

In [ ]:
seif_pattern = re.compile(r"(?:סעיף|ס'|סעיפים)\s{0,2}[\d()אבגדהוזחטיכלמנסעפצקרשת]+")

In [ ]:
df["seifim"] = df["הוראות החיקוק שפורטו בהסדר"].apply(seif_pattern.findall)

In [ ]:
df.drop(df.index[df["seifim"].apply(lambda s: s==[])], inplace = True) # we drop all other we didnt clean

In [ ]:
df["seifim"].apply(lambda s: s==[]).value_counts()  #we will work with 501 enteries (from original 507)

In [ ]:
#cleanfunction for all groups
def clenupDict(seifimitems):
 dictionary=dict()
 for k,v in seifimitems.items():
  new_key=(re.sub(r"\([^)]+\)|\(\d|[\אבגדהוזחטיכלמנסעפצקרשת]+", "", k))  #maybe remove here more
  dictionary.update({new_key:v})
 return (dictionary)

In [ ]:
X = df["seifim"].apply(lambda seifim: {s.strip(" סעיפים סעיף' """):1 for s in seifim })  #coloumn is the seifim
X_new=X.apply(clenupDict)
vectorizer = DictVectorizer(sparse=False)
vectorizer.fit(X_new)
df_seifim = pd.DataFrame(vectorizer.transform(X_new), columns=vectorizer.feature_names_)

In [ ]:
#now we have several coloumn with the samename lets aggrigate them
df_seifim.groupby(df_seifim.columns, axis=1).agg(np.max)

In [ ]:
df_conditional_order = df.reset_index()\
  .join(df_seifim)\
  .rename(columns={"תיאור העובדות המהוות עבירה שבהן הודה החשוד": "facts", "מספר תיק":"id"})\
  .set_index("id")\
  .drop(["הוראות החיקוק שפורטו בהסדר", "נימוקים משתנים לסגירת התיק בהסדר", "תנאי ההסדר", "seifim", "יחידה"],axis=1)
df_conditional_order.head()

In [ ]:
def cleanfacts(dataseries):  #clean data only dates, we can remove numbers as weel??
   return re.sub(r'בתאריך|ביום|או|[0-9]+.[0-9]+.[0-9]|בשנת|([0-9]{2}\.[0-9]{2}\.[0-9]{4})|([0-9]{2}\/[0-9]{2}\/[0-9]{4})+',"",dataseries)

In [ ]:
df_conditional_order["clean_facts"] = df_conditional_order["facts"].apply(cleanfacts)

In [ ]:
vectorizer = CountVectorizer(binary=True)  #thisapproch isnt sogood, cause the correlation should be low
X = vectorizer.fit_transform(df_conditional_order["clean_facts"])
cols = [k for k,v in sorted(vectorizer.vocabulary_.items(), key= lambda t:t[1])]
X = pd.DataFrame(X.todense(), columns=cols)

In [ ]:
#option 1-intuitive(like inclass, just cleaner data)
y = df_conditional_order["415"] 
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=20191120)

model = RandomForestClassifier(random_state=10, n_estimators=100,max_depth=5)
model.fit(X_train, y_train)
print ("RandomForest accuracy in train:",accuracy_score(model.predict(X_train), y_train))
print ("RandomForest accuracy in test:",accuracy_score(model.predict(X_test), y_test))

In [ ]:
#option 1-intuitive(all)
#y = df_conditional_order["415"] 
y=df_conditional_order[["361","338","379","3","60","420","418","348","415"]]
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=20191120)

model = RandomForestClassifier(random_state=5, n_estimators=100,max_depth=5)
model.fit(X_train, y_train)
print ("RandomForest accuracy in train:",accuracy_score(model.predict(X_train), y_train))
print ("RandomForest accuracy in test:",accuracy_score(model.predict(X_test), y_test))

In [ ]:
def get_category():
  lst=[]
  new_df_cat=df_conditional_order.drop(['facts','',"clean_facts"], axis=1)
  for i,row in new_df_cat.iterrows():
    if (1 in row.values):
      val=(row.iloc[row.values==1].index[0])
      #for simplicity willonly ove one category per text
      #s=[]
      #for v in val:
       # s.append(v)
      lst.append(val)
  lst.append('Nan')
  return lst

In [ ]:
#option 2- try create our category according to the map and run using the models
df_conditional_order['category']=get_category()    

In [ ]:
df_conditional_model_order =df_conditional_order[['clean_facts','category']]  #this will be our initiative data set for the models

In [ ]:
#create the entire coloumns to one with  one category
unique_cat = set(df_conditional_model_order['category'].unique())
df_conditional_model_order['category'] = pd.Categorical(df_conditional_model_order['category'], categories=unique_cat).codes

In [ ]:
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=20191120)

In [ ]:
#cosine matric
k=30
model = KNeighborsClassifier(n_neighbors=k,metric='cosine')
model.fit(X_train, y_train)
#df_conditional_model_order['KNN_Pred'] = model.predict(X)

print ("knn accuracy in train:",accuracy_score(model.predict(X_train), y_train))
print ("knn accuracy in test:",accuracy_score(model.predict(X_test), y_test))

In [ ]:
k=10
model = KNeighborsClassifier(n_neighbors=k,metric='hamming')
model.fit(X_train, y_train)
#df_conditional_model_order['KNN_Pred'] = model.predict(X)

print ("haming knn accuracy in train:",accuracy_score(model.predict(X_train), y_train))
print ("haming knn accuracy in test:",accuracy_score(model.predict(X_test), y_test))

In [ ]:
#now lets try RandomForest
model = RandomForestClassifier(random_state=3, n_estimators=100,max_depth=3)
model.fit(X_train, y_train)
print ("RandomForest accuracy in train:",accuracy_score(model.predict(X_train), y_train))
print ("RandomForest accuracy in test:",accuracy_score(model.predict(X_test), y_test))

In [ ]:
#option3- create our own transformer for the text
class CustomEncoderTransformer(BaseEstimator, TransformerMixin):
    def __init__(self,cat,mapped_dic):#,maybe create here the dic according to input dic 
        TransformerMixin.__init__(self)
        self.cat=cat
        self.cat_dict=mapped_dic
    def fit(self, X, y=None):
        return self
    def transform(self, X):
      lst=[]
      for text in X:
        v=self.cat_dict[self.cat]
        r=re.compile(v)
        if len(r.findall(text))>1:
          lst.append(1)
        else:
          lst.append(0)
      return(lst)
      def fit_transform(self,X):
        return self.transform(self,X)

In [ ]:
#ppl = Pipeline([
#    ("my_encoder", CustomHotEncoder())
#])
#ppl.fit(df_conditional_order["clean_facts"], df_conditional_model_order['category'])
#created it according to the number of enteriesfromthe text and the seif, took only 9 features
mapped_dic={"415":r"(?:כזב|מרמה|קבלת דבר במרמה|שווא)","348":r"(?:נשיקה|נשקה|מעשה מגונה|הסכמתה|ללא הסכמה|סיפוק מיני|גירוי|מעשים מגונים|לבוש מפירים|איבר מינה)","418":r"(?:זיוף|זיוף תרופות|מזויפת)","420":r"(?:שימוש במסמך מזויף|מסמך מזויף|ערבות מזויפת|זייף המחאה|המחאה|)","60":r"(?:סימן|סימן מסחר|סימני מסחר|בלא רשות)","3":r"(?:הטרדה מינית|סחיטה|מעשים מגונים|מיני)","379":r"(?:מנע|נותר לעמוד|דחף)","338":r"(?:פזיזות|רשלנות|נפיץ|מותירה)","361":r"(?:|נטישה||בלא השגחה|ללא השגחה|משוטט לבדו)"}

features_cat=["415","348","418","420","60","3","379","338","361"] 
for category in features_cat:
  #my_CustomHotEncoder = CustomEncoderTransformer(category)
  name="category "+str(category)
  my_CustomHotEncoder = CustomEncoderTransformer(category,mapped_dic)
  df_conditional_order[name]=my_CustomHotEncoder.fit_transform(df_conditional_order["clean_facts"])

In [ ]:
#now lets test
X_n=df_conditional_order[["category 415"]]
y = df_conditional_order["415"] 


X_train, X_test, y_train, y_test = \
    train_test_split(X_n, y, train_size=0.7, test_size=0.3, random_state=20191120)

model = RandomForestClassifier(random_state=100, n_estimators=10,max_depth=20)#100,100,10
model.fit(X_train, y_train)
print ("RandomForest accuracy in train:",accuracy_score(model.predict(X_train), y_train))
print ("RandomForest accuracy in test:",accuracy_score(model.predict(X_test), y_test))

In [ ]:
#now lets test, only for 348
X_n=df_conditional_order[["category 348"]]
y = df_conditional_order["348"] 
#y=df_conditional_model_order['category']
#print(X_n)
#print(y)

X_train, X_test, y_train, y_test = \
    train_test_split(X_n, y, train_size=0.7, test_size=0.3, random_state=20191120)

model = RandomForestClassifier(random_state=100, n_estimators=10,max_depth=10)#100,100,10
model.fit(X_train, y_train)
print ("RandomForest accuracy in train:",accuracy_score(model.predict(X_train), y_train))
print ("RandomForest accuracy in test:",accuracy_score(model.predict(X_test), y_test))

In [ ]:
#now lets test, only for 418 and soonforeach one
X_n=df_conditional_order[["category 418"]]
y = df_conditional_order["418"] 
#y=df_conditional_model_order['category']
#print(X_n)
#print(y)

X_train, X_test, y_train, y_test = \
    train_test_split(X_n, y, train_size=0.7, test_size=0.3, random_state=20191120)

model = RandomForestClassifier(random_state=10, n_estimators=5,max_depth=5)#100,100,10
model.fit(X_train, y_train)
print ("RandomForest accuracy in train:",accuracy_score(model.predict(X_train), y_train))
print ("RandomForest accuracy in test:",accuracy_score(model.predict(X_test), y_test))

In [ ]:
#now lets test, all
X_n=df_conditional_order[["category 361","category 338","category 379","category 3","category 60","category 420","category 418","category 348","category 415"]]
#y = df_conditional_order["361"] 
y=df_conditional_model_order['category']


X_train, X_test, y_train, y_test = \
    train_test_split(X_n, y, train_size=0.7, test_size=0.3, random_state=20191120)

model = RandomForestClassifier(random_state=100, n_estimators=50,max_depth=50)#100,100,10
model.fit(X_train, y_train)
print ("RandomForest accuracy in train:",accuracy_score(model.predict(X_train), y_train))
print ("RandomForest accuracy in test:",accuracy_score(model.predict(X_test), y_test))

In [ ]:
#now lets test, all
X_n=df_conditional_order[["category 361","category 338","category 379","category 3","category 60","category 420","category 418","category 348","category 415"]]
y=df_conditional_order[["361","338","379","3","60","420","418","348","415"]]

X_train, X_test, y_train, y_test = \
    train_test_split(X_n, y, train_size=0.7, test_size=0.3, random_state=20191120)

model = RandomForestClassifier(random_state=10, n_estimators=50,max_depth=5)#100,100,10
model.fit(X_train, y_train)
print ("RandomForest accuracy in train:",accuracy_score(model.predict(X_train), y_train))
print ("RandomForest accuracy in test:",accuracy_score(model.predict(X_test), y_test))


In [77]:
#appendix- lets try correlations ,lets see if we have correlation betweenchildneglect to summer
#lets firts find what is summer
from datetime import datetime
def get_summer_time(data):
  date_pattren=re.compile(r"[0-9]{2}\.[0-9]{2}\.[0-9]{4}|[0-9]{2}\/[0-9]{2}\/[0-9]{4}")
  v=date_pattren.findall(data) 
  if len(v)>0:  
    dd=v[0]
    try:
      results=datetime.strptime(v[0],'%d/%m/%Y').month
    except ValueError:
      results=datetime.strptime(v[0],'%d.%m.%Y').month
    if ((results>=5) and (results<=10)):  #summer
      return True
    else:
      return False
  else:
   return False


In [ ]:
df_conditional_order['summer']=df['תיאור העובדות המהוות עבירה שבהן הודה החשוד'].apply(get_summer_time)

In [ ]:
X_n=df_conditional_order[["361"]]  #ילדים באוטו
y=df_conditional_order['summer']

X_train, X_test, y_train, y_test = \
    train_test_split(X_n, y, train_size=0.7, test_size=0.3, random_state=20191120)

model = RandomForestClassifier(random_state=10, n_estimators=10,max_depth=10)#100,100,10
model.fit(X_train, y_train)
print ("RandomForest accuracy in train:",accuracy_score(model.predict(X_train), y_train))
print ("RandomForest accuracy in test:",accuracy_score(model.predict(X_test), y_test))
print (classification_report(y_true=y,y_pred=model.predict(X_n)))

In [ ]:
#an example
df_summer = pd.concat([
  df_conditional_order[["summer", "361"]].reset_index(),
  pd.DataFrame(model.predict(X_n),columns=["predicted"])],
  axis=1
).rename(columns={"summer": "summer_actual"})
#df_summer

In [ ]:
df_summer[(df_summer.summer_actual==df_summer.predicted) & (df_summer.summer_actual==0) & (df_summer["361"]==1)]  #הזנחת ילדים לא קרה בקיץ!!